In [1]:
import os
%pwd

'f:\\TextSummerizer\\research'

In [2]:
os.chdir("f:/TextSummerizer")

In [3]:
%pwd

'f:\\TextSummerizer'

## basic configuration

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [11]:
from src.TextSummerizer.constants import *
from src.TextSummerizer.utils.common import read_yaml, create_directories

## Configuration Updates

In [16]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config['artifacts_root']])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=str(config.source_URL),
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [17]:
import os
import urllib.request as request
import zipfile
from src.TextSummerizer.logging import logger


## components

In [18]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"file downloaded and saved at {filename} with info : {headers}")
        else:
            logger.info(f"file already exists")

    def extract_zip_file(self):
        """
        zip_file_path:str
        extracts the zip file into the data directory
        function returns None
        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [19]:
config=ConfigurationManager()
data_ingestion_config=config.get_data_ingestion_config()
data_ingestion=DataIngestion(data_ingestion_config)

data_ingestion.download_file()
data_ingestion.extract_zip_file()

2025-09-03 15:50:09,766 - INFO - yaml file :config\config.yaml loaded successfully
2025-09-03 15:50:09,769 - INFO - yaml file :F:\TextSummerizer\params.yaml loaded successfully
2025-09-03 15:50:09,769 - INFO - created directory at :artifacts
2025-09-03 15:50:09,769 - INFO - created directory at :artifacts/data_ingestion
2025-09-03 15:50:27,816 - INFO - file downloaded and saved at artifacts\data_ingestion\summarizer-data.zip with info : Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A5EC:47C4A:BDAC9:1CEA13:68B8165F
Accept-Ranges: bytes
Date: Wed, 03 Sep 2025 10:20:16 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830044-CCU
X-Cache: MISS
X-